# Getting Started

Figaro's prompt templates uses Jinja 2 behind the scenes. Which means, you can
use variables, loops, conditions, within a prompt! Figaro also allows you to
chain prompts together in a simple way.

## Simple Prompt Example

This example we send a simple prompt to Vertex AI. We want to get the capital
of California.

In [2]:
# Complete (but simple) Figaro example.
import figaro

template = """
What is the capital of {{us_state}}?
{% gen vertexai "capital" model="text-bison" %}
"""

# Create a Figaro chain.
chain = figaro(template=template, verbose=True)

# Execute the chain with "California" as the input.
result = chain(us_state="California")

print(result)

INFO:root:What is the capital of California?
INFO:root:Sacramento is the capital of California.
INFO:root:
Sacramento is the capital of California.


```jinja2
What is the capital of {{us_state}}?
```

The variable `{{us_state}}` is replaced with the US state passed in when you
run Figaro: `chain(us_state="California")`.

```jinja2
{% gen vertexai "capital" model="text-bison" %}
```

The following line sends everything above it to Vertex AI. `gen` begins the
generative command. `vertexai` tells Figaro what AI to use. 

Next, `"capital"` is the variable where the output will go. It's not used in
this example, but you'll see later what you can do with it.

Finally, `model="text-bison"` is an argument selects which Vertex AI model 
to use.

## Prompt Chaining Example

A prompt chain is when you ask the AI one question then you ask it another 
question based on the previous answer.

Without Figaro you might do something like this:

```py
prompt1 = "What is the capital of California?
answer1 = predict(prompt1)
prompt2 = "What is " + answer1 + "'s population?
answer2 = predict(prompt2)
```

In [11]:
template = """
What is the capital of {{us_state}}? Only return the name of the city.
{% gen vertexai "capital" model="text-bison" %}

What is {{capital}}'s current population? Only return a number.
{% gen vertexai "population" model="text-bison" %}
"""

# Create a Figaro chain.
chain = figaro(template=template, verbose=True)

# Execute the chain with "California" as the input.
result = chain(us_state="California")

print(result)

INFO:root:What is the capital of California? Only return the name of the city.
INFO:root:The capital of California is Sacramento.
INFO:root:What is The capital of California is Sacramento.'s current population? Only return a number.
INFO:root:498,352
INFO:root:
498,352


Here you can see we have two prompts. The first asks what is the capital of
California. The second prompt asks for the capital's current population but we
specifically replaced `capital` with the answer from the first question.

Under the hood Figaro is taking the input of the first prompt and it's output,
combining it with the second prompt to get the second output.

First prompt:
```
What is the capital of California? Only return the name of the city.
```

First answer:
```
Sacramento
```

Second prompt:
```
What is the capital of California? Only return the name of the city.
Sacramento
What is Sacramento's current population? Only return a number.
```

Second answer:
```
493,784
```

Finally, `chain()` will return the output from the last varaible `population`.

Why use Figaro? Traditionally you might do something like this:

```py
def call_vertex_ai(prompt):
  model = TextGenerationModel.from_pretrained("text-bison")
  parameters = { 'temperature': 0.2, "max_tokens": 1024 }
  return model.predict(prompt, **parameters)

state = "California"
prompt1 = "What is the capital of " + state + "?"
answer1 = call_vertex_ai(prompt1)
prompt2 = "What is " + answer1 + 's current population?"
answer2 = call_vertex_ai(prompt2)
print(answer2)
```

This might be fine for this example but concatting strings, loops, and branch
conditions start making the code harder to understand.